In [ ]:
##PLEASE UPVOTE IF YOU FIND IT USEFUL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
train= pd.read_csv('../input/hackerearths-reduce-marketing-waste/train.csv')
test = pd.read_csv('../input/hackerearths-reduce-marketing-waste/test.csv')
test_sub=pd.read_csv('../input/hackerearths-reduce-marketing-waste/test.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe(include='all')

In [ ]:
train.columns

In [ ]:
#replacing $ sign in deal_value and Weighted_amount column and changing datatype
train['Deal_value']=train['Deal_value'].apply(str).str.replace('$','')
train['Weighted_amount']=train['Weighted_amount'].apply(str).str.replace('$','')
train['Deal_value']=train['Deal_value'].astype('float')
train['Weighted_amount']=train['Weighted_amount'].astype('float')

In [ ]:
#testdata
#replacing $ sign in deal_value and Weighted_amount column and changing datatype
test['Deal_value']=test['Deal_value'].apply(str).str.replace('$','')
test['Weighted_amount']=test['Weighted_amount'].apply(str).str.replace('$','')
test['Deal_value']=test['Deal_value'].astype('float')
test['Weighted_amount']=test['Weighted_amount'].astype('float')

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
#CREATE HTML REPORT OF DATA Using the pandas Profiler
profile = ProfileReport(train)
profile

## Null value Treatment

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
#null value treatment
train['Deal_value']=train['Deal_value'].fillna((train['Deal_value'].mean()))
train['Weighted_amount']=train['Weighted_amount'].fillna((train['Weighted_amount'].mean()))
train['Industry']=train['Industry'].fillna((train['Industry'].mode()[0]))
train['Last_lead_update']=train['Last_lead_update'].fillna((train['Last_lead_update'].mode()[0]))
train['Resource']=train['Resource'].fillna((train['Resource'].mode()[0]))


In [ ]:
# As here we split the Location into place and state as Geography feature having
# USA has place and state both in Location feature while in case of India it is missing

train[['place','state']]=train['Location'].str.split(' ',1, expand=True)
train['state'].fillna('0',inplace=True)
train['Geography'].fillna('USA',inplace=True)
train.loc[train['state']=='0', 'Geography'] = 'India'
train.drop(columns=['place','state'], inplace=True)

In [ ]:
#null value treatment
test['Deal_value']=test['Deal_value'].fillna((test['Deal_value'].mean()))
test['Weighted_amount']=test['Weighted_amount'].fillna((test['Weighted_amount'].mean()))
test['Industry']=test['Industry'].fillna((test['Industry'].mode()[0]))
test['Last_lead_update']=test['Last_lead_update'].fillna((test['Last_lead_update'].mode()[0]))
test['Resource']=test['Resource'].fillna((test['Resource'].mode()[0]))


In [ ]:
test[['place','state']]=test['Location'].str.split(' ',1, expand=True)
test['state'].fillna('0',inplace=True)
test['Geography'].fillna('USA',inplace=True)
test.loc[test['state']=='0', 'Geography'] = 'India'
test.drop(columns=['place','state'], inplace=True)

In [ ]:
train.isna().sum()

In [ ]:
train.drop(train.loc[train['Success_probability']>100].index, inplace = True)

In [ ]:
#Filling all the null value with the most occuring one
train_most_common_imputed = train.apply(lambda x: x.fillna(x.value_counts().index[0]))

In [ ]:
#label Encoding
le= preprocessing.LabelEncoder()
for column in train_most_common_imputed.columns:
    if train_most_common_imputed[column].dtype==object:
        train_most_common_imputed[column]=le.fit_transform(train_most_common_imputed[column])
    else:
        pass

In [ ]:
test = test.apply(lambda x: x.fillna(x.value_counts().index[0]))

In [ ]:
#label Encoding Test Data
le= preprocessing.LabelEncoder()
for column in test.columns:
    if test[column].dtype==object:
        test[column]=le.fit_transform(test[column])
    else:
        pass

In [ ]:
 #Selecting best featues according to importance
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
train_most_common_imputed[train_most_common_imputed < 0] = 0
X = train_most_common_imputed.iloc[:,0:]  
y = train_most_common_imputed.iloc[:,-1] 

y=y.astype('int')
bestfeatures = SelectKBest(score_func=chi2, k=16)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Features','Score']  #naming the dataframe columns
print(featureScores.nlargest(23,'Score')) 

In [ ]:
X.drop(columns=['Deal_title','Lead_name','Date_of_creation','Contact_no','POC_name',
       'Lead_POC_email','Success_probability'],inplace=True)

In [ ]:
test.drop(columns=['Deal_title','Lead_name','Date_of_creation','Contact_no','POC_name',
       'Lead_POC_email'],inplace=True)

In [ ]:
#BOXPLOT FOR OUTLIAR DETECTION ON TRAINING DATA
a=X['Weighted_amount']
b=X['Fund_category']
c=X['Geography']
d=X['Lead_source']
e=X['Level_of_meeting']
f=X['Internal_rating']
to_plot=[a,b,c,d,e,f]
fig=plt.figure(1,figsize=(16,9))
ax=fig.add_subplot(111)
bp=ax.boxplot(to_plot)

In [ ]:
#removing Outliars from training data
from scipy import stats
z_scores = stats.zscore(X)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
X = X[filtered_entries]
X

In [ ]:
X_train,X_test,Y_train,Y_test= train_test_split(X,y,test_size=.2,random_state=121)

In [ ]:
from catboost import CatBoostRegressor 
from sklearn.metrics import mean_absolute_error
model_cat= CatBoostRegressor(iterations=2000,random_state=121,learning_rate=.1,max_depth=3)
model_cat.fit( X_train, Y_train, use_best_model=True, eval_set=(X_test, Y_test), plot=True )


In [ ]:
Y_pred_test=model_cat.predict(X_test)
Y_pred=model_cat.predict(test)


In [ ]:
from sklearn import metrics
score=max(0,100-np.sqrt(metrics.mean_squared_error(Y_test,Y_pred_test)))
print(score)

## HyperParameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint

mod= CatBoostRegressor()

par={'max_depth':[3,5,10],
    'n_estimators':[100,200,300,400]
     ,'learning_rate':[0.1,0.01,0.001]}
def hyperparameter_tuning(mod,param_d,p,q):
    rdmsearch=  GridSearchCV(mod, param_grid=param_d,n_jobs=-1,cv=9,scoring='neg_mean_squared_error')
    rdmsearch.fit(p,q)
    ht_params = rdmsearch.best_params_
    ht_score = rdmsearch.best_score_
    return ht_params, ht_score

rf_parameters, rf_ht_score = hyperparameter_tuning(mod, par,  X_train, Y_train)


In [ ]:
print(rf_parameters, rf_ht_score)

In [ ]:
test.head()

In [ ]:
y_test_title=test_sub.Deal_title.values

In [ ]:
sub=pd.DataFrame(y_test_title,columns=['Deal_title'])

In [ ]:
sub['Success_probability']=Y_pred

In [ ]:
sub.head()

In [ ]:
sub.to_csv('sub1.csv',index=False)